In [1]:
!pip install pymongo
!pip install cssselect
import re
import time
from typing import Iterator
import requests
import lxml.html
from pymongo import MongoClient

def main():
    """
    crawlerのメイン処理
    """
    client = MongoClient('localhost', 27017)
    collection = client.scraping.ebooks #scrapingデータベースの、ebooksコレクションを取ってくる
    collection.create_index('key', unique = True) #key fieldにuniqueなindexを作成する

    session = requests.Session()
    res = session.get('https://gihyo.jp/dp')
    urls = scrape_list_page(res)
    for url in urls:
        key = extract_key(url) #urlからキーを取得する
        ebook = collection.find_one(
            {'key': key}
        ) #DBから、keyに該当するデータを探す。
        if ebook is None: #存在しない場合だけ、詳細ページをクロールする
            time.sleep(1)
            res = session.get(url)
            ebook = scrape_detail_page(res)
            collection.insert_one(ebook)
        print(ebook)

def scrape_list_page(res):
    """
    一覧ページ（元）から、詳細ページのurlを抜き出す
    """
    html = lxml.html.fromstring(res.text) #.fromstring()関数
    html.make_links_absolute(res.url) #引数のurlに対しての絶対リンクを作る
    a_link = html.cssselect('#listBook > li > a[itemprop="url"]')
    for a in a_link:
        url = a.get('href')
        yield url

def scrape_detail_page(res):
    """
    詳細ページのresから書籍の情報を取得、dictに格納する
    """
    html = lxml.html.fromstring(res.text)
    ebook = {
        'url': res.url, #url
        'key': extract_key(res.url), #urlからキーを抜き出す
        'title': html.cssselect('#bookTitle')[0].text_content(),
        'price': html.cssselect('.buy')[0].text.strip(),
        'content': [normalize_spaces(h3.text_content()) for h3 in html.cssselect('#content > h3')]
    }
    return ebook

def extract_key(url):
    """
    urlからキーを抜き出す
    """
    m = re.search(r'/([^/]+)$', url)
    return m.group(1)

def normalize_spaces(s):
    """
    連続する空白を一つのスペースに置き換え、前後の空白を削除した新しい文字列を取得する。
    """
    return re.sub(r'\s+', ' ', s).strip()

if __name__ == '__main__':
    main()

{'_id': ObjectId('62a9c95d918df2dec06e2625'), 'url': 'https://gihyo.jp/dp/ebook/2022/978-4-297-12863-0', 'key': '978-4-297-12863-0', 'title': 'スマホではじめるビデオ会議 Zoom & Microsoft Teams［iPhone & Android対応版］', 'price': '1,200円', 'content': ['第1章 ビデオ会議の基本', '第2章 Zoomでビデオ会議に参加しよう', '第3章 Zoomでビデオ会議を開こう', '第4章 Microsoft Teamsでビデオ会議に参加しよう', '第5章 Microsoft Teamsでビデオ会議を開こう', '第6章 ビデオ会議で困ったときのQ&A']}
{'_id': ObjectId('62a9c95f918df2dec06e2626'), 'url': 'https://gihyo.jp/dp/ebook/2022/978-4-297-12887-6', 'key': '978-4-297-12887-6', 'title': '「微」「積」で鍛える数学的思考～分けて集める本当の意味がわかる～', 'price': '2,400円', 'content': ['第1章 数学的思考論', '第2章 微分法の基礎の基礎', '第3章 微分法の基礎・基本', '第4章 微分法の発展', '第5章 微分法の応用', '第6章 積分法の基礎', '第7章 積分法の基本', '第8章 積分法の発展', '第9章 積分法の応用', '第10章 微分法と積分法のさらなる応用']}
{'_id': ObjectId('62a9c960918df2dec06e2627'), 'url': 'https://gihyo.jp/dp/ebook/2022/978-4-297-12859-3', 'key': '978-4-297-12859-3', 'title': '図解即戦力工作機械業界のしくみとビジネスがこれ1冊でしっかりわかる教科書', 'price': '1,500円', 'content': ['Chapter 1 ものづくりを支える工作機械', 'Chapte